In [ ]:
#標點符號、停用詞的移除
#若為空值，則以*取代之
#label重新編碼：0:CNUG, 1:CNUG *, 2:CNUG0, 3:CNaN, 4:HNUG, 5:HNUG *, 6:HNaN

In [1]:
import numpy as np
import pandas as pd
import jieba
import re
from sklearn import preprocessing
import csv

In [2]:
trainset = pd.read_excel('dev+train_clean.xlsx', encoding='utf-8')
trainset[:3]

,id,round,sender,utterance,n,n1,n2,n3,n4,n5,...,n10,n11,n12,n13,n14,n15,n16,n17,n18,n19
0,4227729258237820,1,customer,['#内涵段子# 联通你有些皮了！不过为你点赞 @中国联通 @中国联通客服 掌上营业厅（内涵...,CNaN,CNaN,CNUG,CNaN,CNUG0,CNaN,...,CNUG0,CNaN,CNaN,CNaN,CNaN,CNaN,CNaN,CNaN,CNaN,CNaN
1,4227729258237820,2,helpdesk,"['', '小u来了']",HNaN,HNaN,HNUG,HNaN,HNaN,HNaN,...,HNUG,HNaN,HNaN,HNaN,HNaN,HNaN,HNaN,HNaN,HNaN,HNaN
2,4227729258237820,3,customer,['那夸夸你'],CNaN,CNaN,CNUG,CNaN,CNaN,CNaN,...,CNUG,CNaN,CNaN,CNaN,CNaN,CNaN,CNaN,CNaN,CNaN,CNaN


In [3]:
labels = list(trainset['n'])
texts = list(trainset['utterance'])
ID = list(trainset['id'])
ID_round = list(trainset['round'])
sender = list(trainset['sender'])

In [4]:
def get_encoded_label(labels):
    le = preprocessing.LabelEncoder()
    le.fit(["CNUG0", "CNUG*", "HNUG*", "CNUG", "HNUG", "CNaN", "HNaN"]);
    return le.transform(labels)
#0:CNUG, 1:CNUG*, 2:CNUG0, 3:CNaN, 4:HNUG, 5:HNUG*, 6:HNaN

In [5]:
#labels轉數字
labels = get_encoded_label(labels)

In [6]:
seg_texts = []
for line in texts:
    seg_content = ' '.join(jieba.cut(line, cut_all = False))
    seg_texts.append(seg_content)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/6c/zvr3trdx07525nkmnddnhbh00000gn/T/jieba.cache
Loading model cost 0.722 seconds.
Prefix dict has been built succesfully.


In [7]:
def remove_punctuation(line):
    rule = re.compile("[^a-zA-Z0-9\u4e00-\u9fa5]")
    line = rule.sub(' ',line)
    return line

In [8]:
#去除標點符號
texts = []
for line in seg_texts:
    new_line = remove_punctuation(line).split()
    texts.append(new_line)

In [9]:
cn_stopwords = []
with open('cn_stopwords.txt', 'r', encoding='UTF-8') as file:
    for data in file.read().splitlines():
        cn_stopwords.append(data)

In [10]:
#去除停用詞
pp_texts = []
for line in texts:
    line_noSW = []
    for word in line:
        if word not in cn_stopwords:
            line_noSW.append(word)
    pp_texts.append(line_noSW)

In [11]:
#處理pp_texts內為空值的內容(可能是表情符號)，以*取代
for line in pp_texts:
    if line == []:
        line.append("*")

In [12]:
#以空格串起整個句子
new_texts = []
for sentence in pp_texts:
    series_sentence = " ".join(word for word in sentence)
    new_texts.append(series_sentence)

In [13]:
data = {"id":ID, "round":ID_round, "sender":sender, "texts":new_texts, "labels":list(labels)}
dataset = pd.DataFrame(data)

In [14]:
dataset.to_excel('200428_dataset.xlsx', encoding = 'utf_8_sig')